In [1]:

import numpy
import h5py
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
#Load data and process it

filename = "country_lyrics_all.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()[:10000]
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  10000
Total Vocab:  43


In [3]:
seq_length = 100 # can be changed later
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i: i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
    if i % 100000 == 0: 
        print('here')

n_patterns = len(dataX)
# print(dataX)
print(len(dataX[0]))
print ("Total Patterns: ", len(dataX))

here
100
Total Patterns:  9900


In [4]:

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
print(len(dataX), n_patterns, seq_length, 1)
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(len(y))

9900 9900 100 1
9900


In [5]:
print(X.shape) #(99900, 100, 1)
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, epochs=20, batch_size=1024) #error: Error when checking target: expected dense_6 to have 3 dimensions, but got array with shape (99900, 48)

print("Done!!!")

(9900, 100, 1)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/20
9900/9900 [==============================] - 169s - loss: 3.1917    
Epoch 2/20
9900/9900 [==============================] - 168s - loss: 3.0110    
Epoch 3/20
9900/9900 [==============================] - 165s - loss: 3.0025    
Epoch 4/20
9900/9900 [==============================] - 166s - loss: 2.9975    
Epoch 5/20
9900/9900 [==============================] - 163s - loss: 2.9912    
Epoch 6/20
9900/9900 [==============================] - 166s - loss: 2.9890    
Epoch 7/20
9900/9900 [==============================] - 162s - loss: 2.9834    
Epoch 8/20
9900/9900 [==============================] - 166s - loss: 2.9815    
Epoch 9/20
9900/9900 [==============================] - 166s - loss: 2.9772    
Epoch 10/20
9900/9900 [=======================

In [ ]:
# weights = model.get_weights()
# numpy.save("jank_weights", weights)
# updated_weights = numpy.load("jank_weights.npy")
# model.set_weights(updated_weights)
# model.fit(X, y, epochs=1, batch_size=128) 


In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
#     print(x.shape)
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
#     print(prediction.shape)
#     print(prediction)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")